In [1]:
import tensorflow as tf
import numpy as np
import keras_tuner as kt
import pandas as pd
import seq_generator
import tensorflow.keras.layers as layers
import tensorflow_datasets as tfds
import json

train_ds , val_ds = seq_generator.get_train_and_val(batch_size=32)
cnn = tf.keras.models.load_model("cnn_base/model")

def build_model(hp):


    inputs = tf.keras.Input([5,260,260,3])
    x = layers.TimeDistributed(cnn)(inputs)

    drop1 = hp.Float("drop1",min_value = 0,max_value = 0.4)

    x = layers.TimeDistributed( layers.Dropout(drop1), name = "drop1") (x)

    hidden_units =  hp.Int("hidden_units",min_value=64, max_value=1028, step=64)

    if hp.Choice("rnn",["lstm","gru"]) == "lstm":
        x = layers.LSTM(hidden_units, name = "lstm")(x)
    else:
        x = layers.GRU(hidden_units, name = "gru") (x)

    drop2 = hp.Float("drop2",min_value = 0,max_value = 0.4)
    x = layers.Dropout(drop2,name ="drop2")(x)

    outputs = tf.keras.layers.Dense(2, activation="softmax", name="pred",kernel_regularizer=tf.keras.regularizers.L2())(x)

    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")

    model = tf.keras.Model(inputs,outputs)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss="binary_crossentropy",
        metrics=["accuracy"],
    )

    return model

build_model(kt.HyperParameters())


c:\Users\ual\.conda\envs\felix\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:


!mkdir tuner_run


tuner = kt.Hyperband(build_model,
                     objective='val_accuracy',
                     max_epochs=15,
                     factor=3,
                     directory='tuner_run',
                     project_name='vd_seq')

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

tuner.search(train_ds, epochs=200, validation_data=val_ds, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hp = tuner.get_best_hyperparameters()[0]
with open("tuner_run/best_hps.json","w") as f:
    json.dump(best_hp.values,f)

Trial 30 Complete [00h 18m 58s]
val_accuracy: 0.9120535850524902

Best val_accuracy So Far: 0.9254464507102966
Total elapsed time: 04h 14m 49s
INFO:tensorflow:Oracle triggered exit


KeyError: 'units does not exist.'

In [41]:
build_model(best_hp).save_weights("tuner_run/hp_op/weights.ckpt")